In [ ]:
!pip install faiss-cpu transformers sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss
import numpy as np


In [ ]:
# Example legal documents
legal_docs = [
    "Under California law, the statute of limitations for breach of a written contract is four years.",
    "An employer must provide reasonable accommodations to employees with disabilities under the ADA.",
    "A tenant has the right to a habitable living environment, including heat, water, and electricity.",
    "Defamation involves a false statement presented as a fact that injures a party's reputation."
]


In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(legal_docs, convert_to_tensor=False)

index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])
index.add(np.array(doc_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def retrieve_relevant_legal_docs(query, k=2):
    query_vec = embedder.encode([query])[0]
    D, I = index.search(np.array([query_vec]), k)
    return [legal_docs[i] for i in I[0]]


In [ ]:
query = "What is the statute of limitations for a contract dispute in California?"
relevant_docs = retrieve_relevant_legal_docs(query)
print("Relevant Legal Docs:\n", relevant_docs)


Device set to use cpu


In [ ]:
# Load a small language model for demo purposes (can use GPT-style if needed)
generator = pipeline('text-generation', model='gpt2', tokenizer='gpt2')
context = ' '.join(relevant_docs)
input_prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
response = generator(input_prompt, max_length=100, do_sample=True)[0]['generated_text']
print("\nGenerated Answer:\n", response)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: Neuromorphic computing mimics the neural structure of the human brain. Neuromorphic chips are energy-efficient and ideal for edge AI.
Question: Who coined the term neuromorphic computing?
Answer: There is a real, scientific debate (with many experts), and it's difficult to define what brain we're doing with the concept, however, if these two people, like Watson, who use the term, don't know how long the term term is going to last before or
